# Get Linked_Services

First, lets make sure we have the needed libraries installed, that is needed to run this notebook

In [ ]:
#%pip install azure-identity azure-synapse-artifacts azure-mgmt-synapse

The script above, only need to run the first time, then the modules are dowloaded and installed.

But we need to define, which environment, we would like to run on:

In [ ]:
# Only one of thees can be active at a time
# environment = 'dev'
# environment = 'test'
environment = 'prod'

First lets get a token we can use

In [ ]:
from azure.identity import InteractiveBrowserCredential
from azure.synapse.artifacts import ArtifactsClient
import os

# Indsæt din Synapse Analytics workspace URL
workspace_url = f"https://synw-insight-{environment}.dev.azuresynapse.net"

# Opret credential ved hjælp af interaktiv login
credential = InteractiveBrowserCredential()

# Opret en ArtifactsClient
client = ArtifactsClient(endpoint=workspace_url, credential=credential)

# Eksempel på at få en bearer token
token = credential.get_token("https://dev.azuresynapse.net/.default")
print("Bearer token:", token.token)

Then use this token to get the Linked Services

In [ ]:
import requests
import json

def fetch_all_linked_services(endpoint, access_token):
    linked_services = []
    url = f"{endpoint}/linkedservices?api-version=2020-12-01"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    while url:
        response = requests.get(url, headers=headers)
        data = response.json()
        linked_services.extend(data.get('value', []))
        url = data.get('nextLink')
    
    return linked_services

# Usage
endpoint = workspace_url  # 'https://synw-insight-prod.dev.azuresynapse.net'
access_token = token.token # Indsæt din aktuelle adgangstoken her
linked_services = fetch_all_linked_services(endpoint, access_token)

# Check if the directory exists, if not, create it
directory = './data/linked_services'
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the linked services to a JSON file
with open(f'{directory}/linked_services_{environment}.json', 'w', encoding='utf-8') as f:
    json.dump(linked_services, f, indent=4)

print(f"Linked Services have been saved to linked_services_{environment}.json")


In [ ]:
# Normalize the data
normalized_ls_data = []

for linked_service in linked_services:
    # Opret en dictionary for hver linked service
    linked_service_info = {
        'name': linked_service.get('name'),
        'id': linked_service.get('id'),
        'type': linked_service.get('properties', {}).get('type'),  # Hent typen fra properties
        'url': linked_service.get('properties', {}).get('typeProperties', {}).get('url'),  # Hent url, hvis det er en del af typeProperties
        'connectionString': linked_service.get('properties', {}).get('typeProperties', {}).get('connectionString'),  # Hent connectionString, hvis det eksisterer
        'baseUrl': linked_service.get('properties', {}).get('typeProperties', {}).get('baseUrl'),  # Hent baseUrl, hvis det eksisterer
    }
    
    normalized_ls_data.append(linked_service_info)

print(json.dumps(normalized_ls_data, indent=4))  # Print de normaliserede data i et pænt JSON-format

# Save the normalized data to a JSON file
with open(f'./data/linked_services/normalized_linked_services_{environment}.json', 'w') as f:
    json.dump(normalized_ls_data, f, indent=4)

print(f"Normalized data has been saved to normalized_linked_services_{environment}.json")


Nu kan vi så teste (simpelt), om vi skulle have nogle "forkerte" linked services i vores pågældende miljø

In [ ]:
# Bestem ikke-tilladte substrings baseret på det aktive miljø
disallowed_substrings = []
if environment == 'dev':
    disallowed_substrings = ['-test', '-prod']
elif environment == 'test':
    disallowed_substrings = ['-dev', '-prod']
elif environment == 'prod':
    disallowed_substrings = ['-dev', '-test']

# Iterer gennem normaliseret liste over linked services
for service in normalized_ls_data:
    name = service.get('name', '')
    id_ = service.get('id', '')
    connection_string = service.get('connectionString', '')

    # Sørg for, at connection_string er en streng, før du udfører tjek
    if connection_string is None:
        connection_string = ''

    # Tjek for uønskede substrings i felterne
    if any(substring in name for substring in disallowed_substrings) or \
       any(substring in id_ for substring in disallowed_substrings) or \
       any(substring in connection_string for substring in disallowed_substrings):
        print(f"Fejl i service '{name}' med id '{id_}': indeholder disallowed substrings.")
    # Jeg er lidt ligeglad med dem der er "korrekte"
    # else:
    #     print(f"Service '{name}' er korrekt konfigureret for miljø '{environment}'.")
